In [ ]:
import lightgbm as lgb
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split

train_x = sparse.load_npz('data_l500/train_x.npz')
res = pd.read_csv('data_l500/res.csv')
train_y = pd.read_csv('data_l500/train_y.csv')
test_x = sparse.load_npz('data_l500/test_x.npz')
tr_x, val_x, tr_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=1111)
del train_x
del train_y

In [ ]:
print(tr_x.shape)
print(val_x.shape)
print(tr_y.shape)
print(val_y.shape)
print(test_x.shape)

In [ ]:
print("LGB test")
clf = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
    max_depth=-1, n_estimators=10000, objective='binary',
    subsample=0.6, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.05, min_child_weight=50
)
clf.fit(tr_x, tr_y, eval_set=[(val_x, val_y)], eval_metric='auc', early_stopping_rounds=100)
res['score'] = clf.predict_proba(test_x)[:, 1]
res['score'] = res['score'].apply(lambda x: float('%.6f' % x))
res.to_csv('data_l500/result/submission05041746.csv', index=False)
clf.booster_.save_model('data_l500/result/05041746_744487_740586')